In [20]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
from selenium.common.exceptions import NoSuchElementException
import pandas as pd

In [21]:
# Set up the Chrome driver
driver = webdriver.Chrome()

# Navigate to the website
url = 'https://www.defenceandsecurity.ca/CANSEC/cansec/List-of-Exhibitors'
driver.get(url)
time.sleep(5) # Wait for the page to fully load

In [22]:
companies_name = []
companies_address = []
contacts_name = []
contacts_title = []
contacts_email = []
contacts_phone = []
companies_website = []
companies_desc = []

In [23]:
num = driver.find_element(By.XPATH, '//*[@id="boothList_paginate"]/span/a[6]').text
pages_num = int(num)

In [24]:
for page in range(0, 3):
    companies = driver.find_elements(By.XPATH, "//table[@id='boothList']/tbody/tr[contains(@class, 'odd') or contains(@class, 'even')]")
    for company in companies:
        # Find the exhibitor and click on its profile link
        try:
            exhibitor = company.find_element(By.XPATH, ".//td[2]/div[2]/a")
            driver.execute_script("arguments[0].click();", exhibitor)
        except NoSuchElementException:
            continue

        # Wait for the profile modal to load
        time.sleep(5)

        # Extract the exhibitor information from the profile modal
        modal_body = driver.find_element(By.CLASS_NAME, 'modal-body')
    
        org_name = modal_body.find_element(By.CLASS_NAME, 'modal-orgname').text.strip()
        address = modal_body.find_element(By.CLASS_NAME, 'modal-address').text.strip()
        contact_name = modal_body.find_element(By.CLASS_NAME, 'modal-name').text.strip()
        title = modal_body.find_element(By.CLASS_NAME, 'modal-title').text.strip()

        try:
            email_element = modal_body.find_elements(By.CLASS_NAME, 'modal-email')
            email = email_element[0].text.split(': ')[1]
        except IndexError:
            email = ""

        try:
            phone = modal_body.find_element(By.CLASS_NAME, 'modal-phone').text.split(': ')[1]
        except IndexError:
            phone = ""

        try:
            website = modal_body.find_element(By.CLASS_NAME, 'modal-web').text.split(': ')[1]
        except IndexError:
            website = ""

        try:
            profile_description = modal_body.find_element(By.CLASS_NAME, 'modal-profile').text.strip()
        except NoSuchElementException:
            profile_description = ""

        # Print the exhibitor information
        companies_name.append(org_name)
        companies_address.append(address)
        contacts_name.append(contact_name)
        contacts_title.append(title)
        contacts_email.append(email)
        contacts_phone.append(phone)
        companies_website.append(website)
        companies_desc.append(profile_description)
        
        # Close the profile modal
        close_button = driver.find_element(By.XPATH, "//button[@class='close']")
        close_button.click()
    
    # Navigate to the next page
    next_btn = driver.find_element(By.XPATH, '//*[@id="boothList_next"]')
    driver.execute_script("arguments[0].click();", next_btn)


In [25]:
driver.quit()


In [26]:
df = pd.DataFrame({'Company Name': companies_name, 'Company Address':companies_address, 'Contact Name':contacts_name,
             'Contact Title':contacts_title, 'Contact Email': contacts_email,
              'Contact Phone':contacts_phone,'Company Website':companies_website,
              'Company Description':companies_desc})

In [27]:
df.head()

,Company Name,Company Address,Contact Name,Contact Title,Contact Email,Contact Phone,Company Website,Company Description
0,3-CI Manufacturing INC.,"5555 51 Avenue SE, Calgary, AB, T2C 3X4",Krestoff Slattery,Senior Aerospace/Defense Executive,kslattery@3-ci.com,+1-403-473-8010,www.3-ci.com,"3CI is a Canadian SME, that manufactures Custo..."
1,AAA Canada Inc,"83 rue turgeons , Saint therese, QC, J7E 3H7",Eric Dion,Vice President & COO,eric.dion@aaa-canada.ca,+1-514-473-8372,www.aaa-canada.ca/home,AAA Canada is an on and off-site manufacturing...
2,Accenture,"45 O'Connor St Suite 1100, Ottawa, ON, K1P 1A4",Pierre Dupont,Accenture,,+1-613-750-5000,www.accenture.ca,Accenture is a leading global professional ser...
3,Accipiter Radar,"576 Hwy 20 W, Fenwick, ON, L0S 1E0",Simon Jerome,Vice President of Sales & Marketing,sales@accipiterradar.com,+1-905-228-6888,www.accipiterradar.com/applications/homeland-s...,Accipiter is a global leader in providing affo...
4,Active Gear Co. of Canada Limited,"201 Snidercroft Road, Concord, ON, L4K2J9",contact@activegearco.com,,,+1-905-669-2292,,Active Gear has established itself as a leader...


In [28]:
df.to_excel('defenceandsecurity.xlsx', index = False)
print('File Created')

File Created
